In [1]:
import os
import tweepy
import pandas as pd
from dotenv import load_dotenv, find_dotenv

env_path = find_dotenv()
load_dotenv(env_path)

api_key = os.getenv('API_KEY')
api_key_secret = os.getenv('API_KEY_SECRET')
bearer_token = os.getenv('BEARER_TOKEN')

In [2]:
client = tweepy.Client(bearer_token=bearer_token)
auth = tweepy.OAuthHandler(api_key, api_key_secret)
api = tweepy.API(auth)

In [3]:
query = '(feliz OR felizmente) lang:pt'
#['word', '"words"', 'word1 OR word2, '@mention', '#hashtag', 'to:account', 'since:year-month-day']
# queries can be 512 characters long

In [4]:
tweets = client.search_recent_tweets(query=query, 
                                     tweet_fields=['author_id','created_at','source','entities'],
                                     max_results=10)

In [5]:
for tweet in tweets.data:
    print(tweet.text)
    print(f"Tweet id : {tweet.id}")
    print(f"Author id : {tweet.author_id}")
    print(f"Date : {tweet.created_at}")
    print(f"Source : {tweet.source}")
    if tweet.entities != None :
        if 'hashtags' in tweet.entities.keys():
            for i in range(len(tweet.entities['hashtags'])):
                hashtags = []
                hashtags.append(tweet.entities['hashtags'][i]['tag'])
            print(f"Hashtags : {hashtags}")
    print('------')

@rebanhodamm A saudade é enorme Rebanho, mas ao mesmo tempo muito feliz e orgulhosa de ver que a Marilinha jamais será esquecida! 🤍
Tweet id : 1506355299107618816
Author id : 1483557837687119874
Date : 2022-03-22 19:41:04+00:00
Source : Twitter for Android
------
RT @8luisfernand: Tem vingança melhor que ser feliz?????????? No seu canto, sem piadinha em rede social. Ser feliz fugindo de confusão, sem…
Tweet id : 1506355293860642817
Author id : 3009112150
Date : 2022-03-22 19:41:03+00:00
Source : Twitter for iPhone
------
RT @RACIONAlSDIZ: Um rolê com os aliados já me faz feliz...
Tweet id : 1506355290345725962
Author id : 1084275597415714816
Date : 2022-03-22 19:41:02+00:00
Source : Twitter for iPhone
------
@aethercanon_ VAI SER FELIZ SIM A CARTINHA FOI A PONTA DO ICEBERG
Tweet id : 1506355288403816448
Author id : 1097144760618442754
Date : 2022-03-22 19:41:02+00:00
Source : Twitter Web App
------
RT @CarlaZambelli38: E hoje é o dia dela!

Aniversário da nossa primeira-dama, Michelle 

In [6]:
def hashtag_list(tweet):
    hashtags = []
    if tweet.entities != None :
        if 'hashtags' in tweet.entities.keys():
            for i in range(len(tweet.entities['hashtags'])):
                hashtags.append(tweet.entities['hashtags'][i]['tag'])
    return hashtags

In [7]:
tweets_search = [[],[],[],[],[]]
for tweet in tweets.data :
    tweets_search[0].append(tweet.text)
    tweets_search[1].append(tweet.author_id)
    tweets_search[2].append(tweet.created_at)
    tweets_search[3].append(tweet.source)
    tweets_search[4].append(hashtag_list(tweet))

In [8]:
tweets_search

[['@rebanhodamm A saudade é enorme Rebanho, mas ao mesmo tempo muito feliz e orgulhosa de ver que a Marilinha jamais será esquecida! 🤍',
  'RT @8luisfernand: Tem vingança melhor que ser feliz?????????? No seu canto, sem piadinha em rede social. Ser feliz fugindo de confusão, sem…',
  'RT @RACIONAlSDIZ: Um rolê com os aliados já me faz feliz...',
  '@aethercanon_ VAI SER FELIZ SIM A CARTINHA FOI A PONTA DO ICEBERG',
  'RT @CarlaZambelli38: E hoje é o dia dela!\n\nAniversário da nossa primeira-dama, Michelle Bolsonaro, uma mulher íntegra e sábia, que nos insp…',
  'RT @sanniebloom: Hoje é o dia dele, do nosso dancer!! Yunho, eu te amo mto e tenho um carinho enorme por ti, que hoje vc aproveite mto seu…',
  '@KethlimMillena Feliz aniversário mozao amo você 😍😍😍😍😍🥰',
  'RT @citou_angel: Tem vingança melhor que ser feliz?? No seu canto, sem piadinha em rede social. Ser feliz fugindo de confusão, sem o objeti…',
  'RT @TH_Ferreira1: Puta que pariu nunca pensei q iria ser tão apaixonado em uma

In [10]:
from nltk.tokenize import word_tokenize
import re

# define a preproc function
def preproc_func(tweet):
    '''Does the preprocessing of the tweets'''

    # stopwords: remove articles, prepositions, conjunctions etc
    stopwords=['rt','a','ah','g','h', 'cá','te','tu','tua','tuas','tém','um','uma','você','vocês','vos',
               'à','às','ao','aos','aquela','aquelas','aquele','aqueles','aquilo','as','até','com',
               'como','da','das','de','dela','delas','dele','deles','depois','do','dos','e','ela','elas',
               'ele','eles','em','entre','essa','essas','esse','esses','esta','eu','foi','fomos','for',
               'fora','foram','forem','formos','fosse','fossem','fui','fôramos','fôssemos', 'isso','isto',
               'já','lhe','lhes','me','mesmo','meu','meus','minha','minhas','muito','na','nas','no','nos',
               'nossa','nossas','nosso','nossos','num','numa','nós','oh','o','os','para','pela','pelas','pelo',
               'pelos','por','qual','quando','que','quem','se','seja','sejam','sejamos','sem','serei','seremos',
               'seria','seriam','será','serão','seríamos','seu','seus','somos','sou','sua','suas','são',
               'só','também','ah','q','g','oh','eh','vc','tbm','também','tambem','voceh','você','voce']

    tweet = tweet.lower() # lowercase

    tweet=re.sub('https?://[A-Za-z0-9./]+','',tweet) # remove links que começam com https?://
    tweet=re.sub('https://[A-Za-z0-9./]+','',tweet) # remove links que começam com https://
    tweet=re.sub('http://[A-Za-z0-9./]+','',tweet) # remove links que começam com http://

    tweet = re.sub(r'@[A-Za-z0-9_]+','',tweet) # remove @mentions
    tweet = re.sub(r'#','',tweet) # remove #hashtags

    tweet = re.sub(r'[^\w\s]','',tweet) # remove remove punctuation
    tweet = re.sub(r'[0-9]','',tweet) # remove numbers

    word_tokens=word_tokenize(tweet) # tokenize

    filtered_tweet = [w for w in word_tokens if not w in stopwords] # remove stopwords

    return filtered_tweet

In [11]:
preproc_tweets=[]
for tweet in tweets_search[0]:
    preproc_tweets.append(preproc_func(tweet))

In [13]:
preproc_tweets[0]

['saudade',
 'é',
 'enorme',
 'rebanho',
 'mas',
 'tempo',
 'feliz',
 'orgulhosa',
 'ver',
 'marilinha',
 'jamais',
 'esquecida']

In [ ]:
#for tweet in tweepy.Paginator(client.search_recent_tweets, query=query,
#                              tweet_fields=['context_annotations', 'created_at'], max_results=100).flatten(limit=1000):
#    print(tweet)